In [26]:
#입력예시 2, json이 아닌 문자열 출력, 리스트로 값을 받음
import pandas as pd
from tqdm import tqdm
import random
import os
from datetime import datetime
import numpy as np
import torch
import torch.nn as nn
import itertools

import FinanceDataReader as fdr
import pandas_market_calendars as mcal
import random
import sys

# sys.argv를 사용하여 종목 이름들을 리스트로 받음
selected_companies_names = sys.argv[1:4]

df_krx = fdr.StockListing('KRX')

from datetime import datetime
import pandas as pd
import FinanceDataReader as fdr

# 현재 날짜를 사용
current_date = datetime.now().strftime('%Y%m%d')
start_date = pd.to_datetime(current_date) - pd.DateOffset(years=1)

# 종목 이름을 종목 코드로 변환하고 등락률을 저장할 데이터프레임 초기화
df_returns = pd.DataFrame()
df_close_price = pd.DataFrame()

# 각 종목에 대해 데이터 불러오고 등락률 가져오기
for company_name in selected_companies_names:
    # 종목 코드 가져오기
    try:
        company_code = df_krx[df_krx['Name'] == company_name]['Code'].values[0]
        df_stock = fdr.DataReader(company_code, start_date, current_date)
        df_returns[company_name] = df_stock['Change']
        df_close_price[company_name] = df_stock['Close']
    except IndexError:
        pass
        
# 결과를 출력 (JSON 형식이 아닌 일반 문자열)
#print(df_returns.to_string())

In [27]:
daily_stock_average_d = []
average_returns = df_returns.mean()
average_returns = average_returns * len(df_returns)
for value in average_returns:
    daily_stock_average_d.append(value)

In [28]:
personal_stock_v = []
std_dev = df_returns.std()
total_days = len(df_returns)
volatility = std_dev * (total_days ** 0.5)
for value in volatility:
    personal_stock_v.append(value)

In [29]:
#df_close_price

In [30]:
p12_correlation = df_close_price[selected_companies_names[0]].corr(df_close_price[selected_companies_names[1]])
p13_correlation = df_close_price[selected_companies_names[0]].corr(df_close_price[selected_companies_names[2]])
p23_correlation = df_close_price[selected_companies_names[1]].corr(df_close_price[selected_companies_names[2]])

KeyError: '-f'

In [31]:
stock_corr = []
stock_corr.append(p12_correlation)
stock_corr.append(p13_correlation)
stock_corr.append(p23_correlation)

NameError: name 'p12_correlation' is not defined

In [32]:
def find_combinations():
    possible_values = [i / 100 for i in range(101)] # 0부터 1까지 0.01 간격으로 조합 추출
    combinations = itertools.product(possible_values, repeat=3)

    valid_combinations = []
    valid_indices = []

    for i, combo in enumerate(combinations):
        if round(sum(combo), 2) == 1.0 and all(0.1 <= x <= 0.7 for x in combo): # 최대 0.7, 최소 0.1은 가지는 조합만 추출
            valid_combinations.append(combo)
            valid_indices.append(i)

    return valid_combinations, valid_indices

result, valid_indices = find_combinations()

In [33]:
port_returns = []

for combo in result:
    port_return = sum(x * er for x, er in zip(combo, daily_stock_average_d))
    port_returns.append(port_return)

In [38]:
#국채
from pykrx import bond

# 사용자에게 날짜를 입력받음
date = current_date

# 입력받은 날짜로 데이터프레임 생성
df = bond.get_otc_treasury_yields(date)

# 국고채 3년의 수익률 데이터만 추출하여 출력
kukko_3year_yield = df.loc['국고채 3년', '수익률'] / 100
formatted_yield = f"{kukko_3year_yield:.3f}"  # 소수점 셋째 자리까지 출력


In [35]:
port_vol = []

for combo in result:
    term1 = sum(x ** 2 * sigma ** 2 for x, sigma in zip(combo, personal_stock_v))
    term2 = 0

    for i in range(len(combo) - 1):
        for j in range(i + 1, len(combo)):
            idx = i * (len(combo) - 1) + j - 1
            corr_idx = min(idx, 2)
            term2 += 2 * combo[i] * combo[j] * personal_stock_v[i] * personal_stock_v[j] * stock_corr[corr_idx]

    port_volatility = (term1 + term2) ** 0.5
    port_vol.append(port_volatility)

IndexError: list index out of range

In [36]:
# government_bond = Rf
Rf = float(government_bond)
Cal_values = []

# port_returns_numeric = [float(val) for val in port_returns]

for i in range(len(port_returns)):
    Cal = (port_returns[i] - Rf) / port_vol[i]
    Cal_values.append(Cal)

NameError: name 'government_bond' is not defined

In [37]:
max_cal_value = max(Cal_values)
max_cal_index = Cal_values.index(max_cal_value)
max_result = result[max_cal_index]

NameError: name 'Cal_values' is not defined

In [39]:
Investment = int(sys.argv[4])
danger = int(sys.argv[5])
#고위험 = sys.argv[5] = 7, 중위험 = sys.argv[5] = 5, 저위험 = sys.argv[5] = 3

invest_amount = []
real_invest = (Investment * (danger * 0.1))
rf = Investment - real_invest
for i in range(len(max_result)):
  price = real_invest * max_result[i]
  invest_amount.append(price)

print("총 투자 금액 : {:,.0f}".format(Investment))
print("rf : {:,.0f}".format(rf))
for i in range(len(selected_companies_names)):
  print("{} 투자 금액: {:,.0f}".format(selected_companies_names[i], invest_amount[i]))



#sys.argv[1:4] = 주식종목 3개
#sys.argv[4] = 총투자금액 (total)
#sys.argv[5] = 고위험,중위험,저위험 
#selected_companies_names = sys.argv[1:4]
#Investment = int(sys.argv[4])
#danger = sys.argv[5] = 7, 중위험 = sys.argv[5] = 5, 저위험 = sys.argv[5] = 3

IndexError: list index out of range